# Model Selection with WBIC

In [1]:
from datetime import timedelta

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import orbit
print(orbit.__version__)
from orbit.models import DLT,ETS, KTRLite, LGT
from orbit.utils.simulation import make_trend, make_regression

1.1.0dev


In [2]:
orbit.__version__

'1.1.0dev'

In [3]:
%load_ext autoreload
%autoreload 2

Generate a regression problem with trend with `8` number of regressors where only `3` of them are effective. First, generate the `3` effective regressors.

In [4]:
NUM_OF_REGRESSORS = 8
NUM_OF_EFFECTIVE_REGRESSORS = 3
SERIES_LEN = 100
SEED = 20210101
# sample some coefficients
COEFS = np.random.default_rng(SEED).uniform(-1, 1, NUM_OF_EFFECTIVE_REGRESSORS)
trend = make_trend(SERIES_LEN, rw_loc=0.01, rw_scale=0.1)
x, regression, coefs = make_regression(series_len=SERIES_LEN, coefs=COEFS)
print(regression.shape, x.shape)

# combine trend and the regression
y = trend + regression
y = y - y.min()

(100,) (100, 3)


We can add `5` irrelevant regressors into the dataset to add challenge in selecting the best model.

In [5]:
x_extra = np.random.normal(0, 1, (SERIES_LEN, NUM_OF_REGRESSORS - NUM_OF_EFFECTIVE_REGRESSORS))
x = np.concatenate([x, x_extra], axis=-1)
print(x.shape)

(100, 8)


In [6]:
x_cols = [f"x{x}" for x in range(1, NUM_OF_REGRESSORS + 1)]
response_col = "y"
dt_col = "date"
obs_matrix = np.concatenate([y.reshape(-1, 1), x], axis=1)
# make a data frame for orbit inputs
df = pd.DataFrame(obs_matrix, columns=[response_col] + x_cols)
# make some dummy date stamp
dt = pd.date_range(start='2016-01-04', periods=SERIES_LEN, freq="1W")
df['date'] = dt
print(df.shape)
print(df.head())

(100, 10)
          y        x1        x2        x3        x4        x5        x6  \
0  3.010526  0.172792  0.000000  0.165219 -0.414958  0.771912 -0.278550   
1  3.448303 -0.000000  0.452678  0.223187  1.288435  1.212709  0.071109   
2  3.012242 -0.000000  0.290559  0.182286  0.360842  1.840824  0.873697   
3  1.422250  0.147066  0.014211  0.273356 -1.094259 -1.299734 -1.274814   
4  3.275717 -0.368227 -0.081455 -0.241060 -1.532786 -0.564161  0.765820   

         x7        x8       date  
0 -0.502871  0.060721 2016-01-10  
1  0.083249  0.056608 2016-01-17  
2 -0.364185 -0.570404 2016-01-24  
3  0.824271  0.161256 2016-01-31  
4 -1.186428 -0.435834 2016-02-07  


In [7]:
 regressor_col = x_cols[:3 + 1]


In [8]:
dlt_mod = DLT(
        response_col=response_col,
        date_col=dt_col,
        regressor_col=regressor_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        slope_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,

    )
dlt_mod.fit_WBIC(df=df) #, , sampling_temperature = np.log(100.0)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_68a14a8546a641d75df59b4752abc6a7 NOW.
In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:771:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969:
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it wit

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:781:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan_fit.hpp:15:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/random/additive_combine.hpp:27:
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/random/linear_congruential.hpp:140:20: warning: overlapping comparisons always evaluate to false [-Wtautological-overlap-compare]
        if(_x <= 0 && _x != 0) {
           ~~~~~~~~^~~~~~~~~~
/Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/boost_1.69.0/boost/random/linear_congruential.hpp:393:11: note: in instantiation of member function 'boost::ran

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

...

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

In file included from /var/folders/2r/w1lk3nxj09s73plknkyzzj5m0000gn/T/pystan_cj9e35ve/stanfit4anon_model_68a14a8546a641d75df59b4752abc6a7_1099061517816913695.cpp:780:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/py_var_context.hpp:12:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/src/stan/io/dump.hpp:6:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1:
In file included from /Users/gavin.steininger/.pyenv/versions/3.7.12/lib/python3.7/site-packages

156 warnings generated.



Gradient evaluation took 0.000218 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.18 seconds.
Adjust your expectations accordingly!



Gradient evaluation took 0.000257 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.57 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)

Gradient evaluation took 0.000227 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.27 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)

Gradient evaluation took 0.000213 seconds
1000 transitions using 10 leapfrog steps per transition would take 2.13 seconds.
Adjust your expectations accordingly!


Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:    1 / 2000 [  0%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  200 / 2000 [ 10%]  (Warmup)
Iteration:  400 / 2000 [ 20%]  (Warmup)
Iteration:  200 / 2000 [

-141.27574689492872

In [ ]:
ets_mod = ETS(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,

    )

ets_mod.fit_WBIC(df)

In [ ]:
KTRLite_mod = KTRLite(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        #level_sm_input=0.01,
        estimator='stan-map',
        #num_warmup=4000,
        #num_sample=4000,

    )

KTRLite_mod.fit(df)

In [ ]:
LGT_mod = LGT(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        #level_sm_input=0.01,
        estimator='stan-mcmc',
        num_warmup=4000,
        num_sample=4000,

    )

LGT_mod.fit_WBIC(df)

In [ ]:
LGT_mod = LGT(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        #level_sm_input=0.01,
        estimator='stan-map',
        num_warmup=4000,
        num_sample=4000,

    )

LGT_mod.fit(df)

In [ ]:
dlt_mod = DLT(
        response_col=response_col,
        date_col=dt_col,
        regressor_col=regressor_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        slope_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,
        estimator='stan-map',

    )
dlt_mod.fit(df=df) #, , sampling_temperature = np.log(100.0)

In [ ]:
ets_mod.fit_WBIC(df=df)

In [ ]:
ets_mod = ETS(
        response_col=response_col,
        date_col=dt_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,

    )
ets_mod.fit(df=df,sampling_temperature = np.log(100.0)) #, , sampling_temperature = np.log(100.0)

In [ ]:
ets_mod.get_WBIC_value()

In [ ]:
ForecasterException: Model class: <class 'orbit.template.dlt.DLTModel'> is incompatible with 
        Estimator: <class 'orbit.estimators.stan_estimator.StanEstimatorMCMC'>.  
            Estimator Support: [<class 'orbit.estimators.stan_estimator.StanEstimatorMAP'>, 
                                <class 'orbit.estimators.stan_estimator.StanEstimatorMCMC'>
                                <class 'orbit.estimators.stan_estimator.StanEstimatorMCMC'>]

Now, we can calculate WBIC and compare them across models.

In [ ]:
%%time
wbics = np.empty(NUM_OF_REGRESSORS)

for idx in range(NUM_OF_REGRESSORS):
    regressor_col = x_cols[:idx + 1]

    dlt_mod = DLT(
        response_col=response_col,
        date_col=dt_col,
        regressor_col=regressor_col,
        seed=2020,
        # fixing the smoothing parameters to learn regression coefficients more effectively
        level_sm_input=0.01,
        slope_sm_input=0.01,
        num_warmup=4000,
        num_sample=4000,
    )
    dlt_mod.fit(df=df)
    wbic = dlt.get_training_metrics()['WBIC']
    print('Regressors:{} WBIC:{:.5f}'.format(regressor_col, wbic))
    wbics[idx] = wbic

We plot the chart with WBICs against number of regressors included.  As we can see, WBIC is lowest when regressors overlapped exactly with the truth.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))
ax.plot(np.arange(1, NUM_OF_REGRESSORS + 1), wbics, color='dodgerblue', label='WBICs')
ax.axvline(x=3, linestyle='--', color='orange', label='truth')
ax.set_xlabel('Number of Regressors')
ax.set_ylabel('WBIC')
ax.set_title('Model Selection with WBIC')
fig.legend()